## Exemple using MLP multi-class with boosting and majority vote

# loading Dataset Iris

In [1]:
import time
import gc
import os
from contextlib import contextmanager
from sklearn import datasets
import numpy as np
import pandas as pd

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print('[{0}] done in {1}'.format(name,time.time() - t0))
    

with timer('Loading Data'):
    iris = datasets.load_iris()
    X = iris.data
    y = iris.target

[Loading Data] done in 0.00124597549438


In [2]:
#transform dataset in dataframe

df_data = pd.DataFrame(data=X,columns=iris.feature_names)
df_trgt = pd.DataFrame(data=y,columns=['target'])
df = pd.concat([df_data,df_trgt],axis=1)
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


# creating estimators

In [17]:
#from sklearn.neural_network import MLPClassifier
from Functions.MLPClassification import MLPSKlearn,MLPhep
#from hep_ml.nnet import MLPClassifier
from hep_ml.nnet import MLPMultiClassifier
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier,VotingClassifier

#mlp = MLPClassifier(hidden_layer_sizes=(40,),
#                    activation='tanh',
#                    solver='adam',
#                    alpha=0.0001, 
#                    batch_size='auto',
#                    learning_rate='constant',
#                    learning_rate_init=0.001,
#                    power_t=0.5,
#                    max_iter=1000,
#                    shuffle=True,
#                    random_state=None,
#                    tol=0.01,
#                    verbose=True,
#                    warm_start=False,
#                    momentum=0.9,
#                    nesterovs_momentum=True,
#                    early_stopping=True,
#                    validation_fraction=0.2,
#                    beta_1=0.9,
#                    beta_2=0.999,
#                    epsilon=1e-08)

#neural = MLPClassifier(layers=[10], trainer='adadelta', trainer_parameters={'batch': 600})
#mlp = MLPMultiClassifier(layers=(10,), scaler='standard', trainer='irprop-', epochs=100,
#                trainer_parameters=None, random_state=None)

mlp = MLPSKlearn()
#mlp = MLPhep()

estimator = 5
classifier = AdaBoostClassifier(base_estimator=mlp,
                                n_estimators=estimator,
                                algorithm='SAMME.R')
#mlp = MLPClassifier()
#mj_vot = VotingClassifier(tuple(classifier.estimators_),n_jobs=-1)

# Pre-processing of dataset

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# fit Ensemble

In [19]:
#Usingo only MLP simple 
#mlp.fit(data_preproc_train,y_train)

#using MLP with boosting
classifier.fit(X_train,y_train)

[+] 1 of 1 inits
Epoch 1/20
120/120 [==============================] - 0s 1ms/step - loss: 0.0030 - acc: 0.0083
Epoch 2/20
120/120 [==============================] - 0s 28us/step - loss: 0.0029 - acc: 0.0083
Epoch 3/20
120/120 [==============================] - 0s 28us/step - loss: 0.0029 - acc: 0.0250
Epoch 4/20
120/120 [==============================] - 0s 27us/step - loss: 0.0028 - acc: 0.0333
Epoch 5/20
120/120 [==============================] - 0s 28us/step - loss: 0.0028 - acc: 0.0500
Epoch 6/20
120/120 [==============================] - 0s 27us/step - loss: 0.0027 - acc: 0.0500
Epoch 7/20
120/120 [==============================] - 0s 27us/step - loss: 0.0027 - acc: 0.0667
Epoch 8/20
120/120 [==============================] - 0s 26us/step - loss: 0.0026 - acc: 0.1167
Epoch 9/20
120/120 [==============================] - 0s 27us/step - loss: 0.0026 - acc: 0.1417
Epoch 10/20
120/120 [==============================] - 0s 27us/step - loss: 0.0026 - acc: 0.1750
Epoch 11/20
120/120 [==

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=MLPSKlearn(activation=None, alpha=None, batch_size=None, beta_1=None,
      beta_2=None, dir=None, early_stopping=None, epsilon=None,
      hidden_layer_sizes=None, learning_rate=None, learning_rate_init=None,
      max_iter=None, momentum=None, n_iter_no_change=None,
      nesterovs_momentum=None, power_t=None, shuffle=None, solver=None,
      tol=None, validation_fraction=None, verbose=None, warm_start=None),
          learning_rate=1.0, n_estimators=5, random_state=None)

In [20]:
len(classifier.estimators_)

5

In [16]:
for est in classifier.estimators_:
    print est.predict(X_test)

[0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [15]:
mj_vot = VotingClassifier(zip(['est{0}'.format(i) for i in range(estimator)],
                              classifier.estimators_))

#using votMaj with boosting MLP
mj_vot.predict(X_train)

NotFittedError: This VotingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.